In [1]:
# Import the requests library and API key
import pandas as pd
import requests
from config import tmdb_api_key

## Create a dataframe to store all LGBTQ movies.

In [2]:
# Starting URL for TMDB API including the API key from config.py
url = "https://api.themoviedb.org/3/discover/movie?api_key=" + tmdb_api_key

In [3]:
# URL for movies tagged with keywords "LGBT, gay, lesbian, trans, and transgender"
# and using the language (English), region (US), release date start (01-01-2000) and end (12-31-2022), 
# sort by (release date descending) parameters. Not included are the "page" and "certification" parameters which
# will be included below.
lgbt_url = url + "&language=en-US&region=US&sort_by=primary_release_date.desc&certification_country=US&include_adult=false&include_video=false&primary_release_date.gte=2000-01-01&primary_release_date.lte=2022-12-31&with_keywords=158718%7C264384%7C264386%7C273637%7C290527%7C163037&with_original_language=en"

In [4]:
# Create a function to iterate through multiple pages of the API call. 
# Specifying the params for params with a dictionary containing the key/value pair for "page".

def get_page(page_num):
    lgbt_movies = requests.get(
        lgbt_url,
        params={"page": page_num}
    )
    # Create a variable to hold the JSON text of the 'get' request
    lgbt_movies_data = lgbt_movies.json()
    # Create a Pandas Dataframe for 'results' from the JSON dictionary
    return pd.DataFrame(lgbt_movies_data['results'])

# The number of pages you want
pages_requested = 65

# Get all pages as dataframes by calling the function "get_page()" where "page_num" is variable n + 1.
# The list comprehension will iterate through the range of "pages_requested" which is 65, adding 1 to
# each iteration to make the count 1-65.
pages = [get_page(n + 1) for n in range(pages_requested)]

# Combine pages to single dataframe using the concat function (aka concatenate)
lgbt_movies_df = pd.concat(pages)

In [5]:
# Verify that the lgbt_movies_df was created correctly with all the necessary results.
print(len(lgbt_movies_df))
lgbt_movies_df.head()

1297


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,None,"[35, 18]",507903,en,Spoiler Alert,The story of Michael Ausiello and Kit Cowan's ...,3.483,/np3y8wm2K7YK5RT6e68hw6IzhBw.jpg,2022-12-31,Spoiler Alert,False,0.0,0
1,False,None,"[27, 53]",876802,en,Saint Drogo,Saint Drogo is a supernatural slowburn themati...,1.070,/ghq9XVBvmlbqrsKA7KDMsYOPjAu.jpg,2022-10-01,Saint Drogo,False,0.0,0
2,False,None,[10749],929477,en,Heart Shot,Teenagers Nikki and Sam are in love and planni...,3.082,None,2022-02-17,Heart Shot,False,0.0,0
3,False,/qQFvUg92Uh1jRXCNgqo9LduBDFD.jpg,[18],920345,en,EMPATHY (Or: The Girl with The Pearled Hair),After running away from her emotionally abusiv...,4.915,/59C1l5dqtKuKuKy5qlE23PV4yC6.jpg,2022-01-25,EMPATHY (Or: The Girl with The Pearled Hair),False,0.0,0
4,False,/rzu9BVI9HqY5rG0Xq05ZhewdkYP.jpg,"[18, 35]",641934,en,Am I Ok?,Lucy and Jane have been best friends for most ...,9.426,None,2022-01-24,Am I Ok?,False,7.8,2


In [6]:
# Check for duplicates
lgbt_movies_df[lgbt_movies_df.id.duplicated()]

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count


## Create dataframes to hold all movies that received a certain MPA rating

In [7]:
# In a similar fashion to the above, create a function to iterate through multiple pages of the API call
# and to constrain the search to particular certification ratings. Specifying the params for params
# with a dictionary containing the key/value pair for "page" and "certification".

def get_ratings(page_num, rating):
    lgbt_movies = requests.get(
        lgbt_url,
        params={
            "page": page_num,
            "certification": rating}
    )
    # Create a variable to hold the JSON text of the 'get' request
    lgbt_ratings_data = lgbt_movies.json()
    # Create a Pandas Dataframe for 'results' from the JSON dictionary
    return pd.DataFrame(lgbt_ratings_data['results'])

# The number of pages you want. (Note: the max number of pages for the ratings comes from "NR" 
# having a max of 12 pages of results, and the largest amount of pages, which is why the variable
# was changed from above).
pages_requested = 12

# Get all pages for each US rating as dataframes by calling the function "get_ratings()" 
# where "page_num" is variable n + 1. The list comprehension will iterate through the range 
# of "pages_requested" which is 12, adding 1 to each iteration to make the count 1-12. The get_ratings function
# uses the second variable to specify the rating needed in the URL parameter.
pages_nr = [get_ratings(n + 1, "NR") for n in range(pages_requested)]
pages_g = [get_ratings(n + 1, "G") for n in range(pages_requested)]
pages_pg = [get_ratings(n + 1, "PG") for n in range(pages_requested)]
pages_pg13 = [get_ratings(n + 1, "PG-13") for n in range(pages_requested)]
pages_r = [get_ratings(n + 1, "R") for n in range(pages_requested)]
pages_nc17 = [get_ratings(n + 1, "NC-17") for n in range(pages_requested)]

# Combine pages for each rating to single dataframe using the concat function (aka concatenate)
lgbt_nr_df = pd.concat(pages_nr)
lgbt_g_df = pd.concat(pages_g) 
lgbt_pg_df = pd.concat(pages_pg)
lgbt_pg13_df = pd.concat(pages_pg13)
lgbt_r_df = pd.concat(pages_r)
lgbt_nc17_df = pd.concat(pages_nc17)

In [8]:
# Verify that the lgbt_r_df was created correctly with all the necessary results. (This will ensure all other
# ratings dataframes were created correctly as well).
print(len(lgbt_r_df))
lgbt_r_df.head()

108


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,None,"[27, 53]",876802,en,Saint Drogo,Saint Drogo is a supernatural slowburn themati...,1.070,/ghq9XVBvmlbqrsKA7KDMsYOPjAu.jpg,2022-10-01,Saint Drogo,False,0.0,0
1,False,/Ainzn99TYR1LZWL7pKEBx3AH8Oy.jpg,[35],915164,en,TWO BEERS,"Two lifelong friends, Graham and Adrian, have ...",0.738,/pXEgPcmE1XRwF6543iOFM2XLGzU.jpg,2021-12-10,TWO BEERS,False,10.0,1
2,False,/nsyyWY7F3wjZT6aCkwWIsZXI4wV.jpg,[99],857731,en,Untold: Caitlyn Jenner,Caitlyn Jenner's unlikely path to Olympic glor...,7.605,/gSOXqj8HLTMzdifHOOQnbjgVEVr.jpg,2021-08-24,Untold: Caitlyn Jenner,False,6.5,24
3,False,/zKD0XxAjtK0VYX7j1elDtSNSNbR.jpg,"[18, 12]",552269,en,Joe Bell,"The true story of a small town, working class ...",10.436,/9HfQD60w6f6PPkfdiI6ipJSw0bD.jpg,2021-07-23,Joe Bell,False,6.0,50
4,False,/5C8bfwglg91uZhc2fbfpSjNGamV.jpg,"[27, 9648]",591273,en,Fear Street: 1994,"After a series of brutal slayings, a teen and ...",171.846,/9J9Wy39ZjrVmfk7yMkulpcI5sy0.jpg,2021-06-28,Fear Street: 1994,False,6.8,1384


### Create a new column to hold rating info for each individual dataframe

In [9]:
# Create a variable to hold the list of US ratings names.
ratings = ["NR", "G", "PG", "PG-13", "R", "NC-17"]

In [10]:
# Add a new column in each specific dataframe to label each movie in the dataframe with their corresponding rating
# (this will be important to have later when joining these dataframes into the larger lgbt_movies_df).
lgbt_nr_df["rating"] = ratings[0]
lgbt_g_df["rating"] = ratings[1]
lgbt_pg_df["rating"] = ratings[2]
lgbt_pg13_df["rating"] = ratings[3]
lgbt_r_df["rating"] = ratings[4]
lgbt_nc17_df["rating"] = ratings[5]

In [11]:
# Verify the length of each dataframe.
# (Note: there are no films in the G rating dataframe, meaning there are no LGBT films with a G rating).)
print(len(lgbt_nr_df))
print(len(lgbt_g_df))
print(len(lgbt_pg_df))
print(len(lgbt_pg13_df))
print(len(lgbt_r_df))
print(len(lgbt_nc17_df))

# Verify the new column was created correctly on one of the dataframes.
lgbt_nr_df.head(1)

225
0
12
47
108
3


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,rating
0,False,/qQFvUg92Uh1jRXCNgqo9LduBDFD.jpg,[18],920345,en,EMPATHY (Or: The Girl with The Pearled Hair),After running away from her emotionally abusiv...,4.915,/59C1l5dqtKuKuKy5qlE23PV4yC6.jpg,2022-01-25,EMPATHY (Or: The Girl with The Pearled Hair),False,0.0,0,NR


In [12]:
# Drop all columns except for "id" and "rating" from the individual rating dataframes. Column "id" will be needed
# later for joining the dataframes to lgbt_movies_df. (Note: the dataframe for "G" ratings is not included as the 
# dataframe is empty as shown above).
lgbt_nr_df = lgbt_nr_df[["id", "rating"]]
lgbt_pg_df = lgbt_pg_df[["id", "rating"]]
lgbt_pg13_df = lgbt_pg13_df[["id", "rating"]]
lgbt_r_df = lgbt_r_df[["id", "rating"]]
lgbt_nc17_df = lgbt_nc17_df[["id", "rating"]]

In [13]:
# Verify "r" rating dataframe dropped the correct columns.
lgbt_r_df.head()

,id,rating
0,876802,R
1,915164,R
2,857731,R
3,552269,R
4,591273,R


### Create singlular dataframe to hold all rated movies.

In [14]:
# Join the individual ratings dataframes into a single ratings dataframe.
lgbt_ratings_df = pd.concat([lgbt_nr_df, lgbt_pg_df, lgbt_pg13_df, lgbt_r_df, lgbt_nc17_df])

# Reset the index and drop the former index column for clearer legibility.
lgbt_ratings_df = lgbt_ratings_df.reset_index().drop(columns='index')
lgbt_ratings_df.tail()

,id,rating
390,95048,R
391,22597,R
392,526034,NC-17
393,94025,NC-17
394,1378,NC-17


In [15]:
# Check for duplicates in ratings dataframe
lgbt_ratings_df[lgbt_ratings_df.id.duplicated()]

,id,rating
328,157370,R
349,39958,R
381,16802,R


In [16]:
# Determine how many duplicates there are of each and why.
lgbt_ratings_df.loc[lgbt_ratings_df.id.isin([157370,39958,16802])]

,id,rating
106,157370,NR
159,39958,NR
216,16802,NR
328,157370,R
349,39958,R
381,16802,R


In [17]:
# Drop duplicates and verify that they correctly dropped.
lgbt_ratings_df = lgbt_ratings_df.drop_duplicates(subset=['id'], keep='last')
lgbt_ratings_df.loc[lgbt_ratings_df.id.isin([157370,39958,16802])]

,id,rating
328,157370,R
349,39958,R
381,16802,R


## Join ratings dataframe into main movies dataframe.

In [18]:
# Set index to be "id" for all dataframes.
lgbt_movies_df = lgbt_movies_df.set_index('id')
lgbt_ratings_df = lgbt_ratings_df.set_index('id')

In [19]:
# Verify that index was set correctly
lgbt_ratings_df.head()

,rating
id,
920345,NR
723377,NR
879943,NR
849934,NR
776586,NR


In [20]:
# Join the ratings dataframe into the movies dataframe to add the certification rating to the movies dataframe.
lgbt_movies_df = lgbt_movies_df.join(lgbt_ratings_df, on='id', how='left')

In [21]:
# Verify the length of the dataframe and that the ratings (for movies that had ratings) were added.
print(len(lgbt_movies_df))
lgbt_movies_df.head()

1297


,adult,backdrop_path,genre_ids,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,rating
id,,,,,,,,,,,,,,
507903,False,None,"[35, 18]",en,Spoiler Alert,The story of Michael Ausiello and Kit Cowan's ...,3.483,/np3y8wm2K7YK5RT6e68hw6IzhBw.jpg,2022-12-31,Spoiler Alert,False,0.0,0,NaN
876802,False,None,"[27, 53]",en,Saint Drogo,Saint Drogo is a supernatural slowburn themati...,1.070,/ghq9XVBvmlbqrsKA7KDMsYOPjAu.jpg,2022-10-01,Saint Drogo,False,0.0,0,R
929477,False,None,[10749],en,Heart Shot,Teenagers Nikki and Sam are in love and planni...,3.082,None,2022-02-17,Heart Shot,False,0.0,0,PG-13
920345,False,/qQFvUg92Uh1jRXCNgqo9LduBDFD.jpg,[18],en,EMPATHY (Or: The Girl with The Pearled Hair),After running away from her emotionally abusiv...,4.915,/59C1l5dqtKuKuKy5qlE23PV4yC6.jpg,2022-01-25,EMPATHY (Or: The Girl with The Pearled Hair),False,0.0,0,NR
641934,False,/rzu9BVI9HqY5rG0Xq05ZhewdkYP.jpg,"[18, 35]",en,Am I Ok?,Lucy and Jane have been best friends for most ...,9.426,None,2022-01-24,Am I Ok?,False,7.8,2,NaN


In [22]:
# Swap null values in ratings column for NR
lgbt_movies_df["rating"].fillna("NR", inplace=True)

In [23]:
# Verify null values were replaced.
lgbt_movies_df.head()

,adult,backdrop_path,genre_ids,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,rating
id,,,,,,,,,,,,,,
507903,False,None,"[35, 18]",en,Spoiler Alert,The story of Michael Ausiello and Kit Cowan's ...,3.483,/np3y8wm2K7YK5RT6e68hw6IzhBw.jpg,2022-12-31,Spoiler Alert,False,0.0,0,NR
876802,False,None,"[27, 53]",en,Saint Drogo,Saint Drogo is a supernatural slowburn themati...,1.070,/ghq9XVBvmlbqrsKA7KDMsYOPjAu.jpg,2022-10-01,Saint Drogo,False,0.0,0,R
929477,False,None,[10749],en,Heart Shot,Teenagers Nikki and Sam are in love and planni...,3.082,None,2022-02-17,Heart Shot,False,0.0,0,PG-13
920345,False,/qQFvUg92Uh1jRXCNgqo9LduBDFD.jpg,[18],en,EMPATHY (Or: The Girl with The Pearled Hair),After running away from her emotionally abusiv...,4.915,/59C1l5dqtKuKuKy5qlE23PV4yC6.jpg,2022-01-25,EMPATHY (Or: The Girl with The Pearled Hair),False,0.0,0,NR
641934,False,/rzu9BVI9HqY5rG0Xq05ZhewdkYP.jpg,"[18, 35]",en,Am I Ok?,Lucy and Jane have been best friends for most ...,9.426,None,2022-01-24,Am I Ok?,False,7.8,2,NR


In [24]:
# Reset index.
lgbt_movies_df = lgbt_movies_df.reset_index()

In [25]:
# Verify index reset correctly.
lgbt_movies_df.head()

,id,adult,backdrop_path,genre_ids,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,rating
0,507903,False,None,"[35, 18]",en,Spoiler Alert,The story of Michael Ausiello and Kit Cowan's ...,3.483,/np3y8wm2K7YK5RT6e68hw6IzhBw.jpg,2022-12-31,Spoiler Alert,False,0.0,0,NR
1,876802,False,None,"[27, 53]",en,Saint Drogo,Saint Drogo is a supernatural slowburn themati...,1.070,/ghq9XVBvmlbqrsKA7KDMsYOPjAu.jpg,2022-10-01,Saint Drogo,False,0.0,0,R
2,929477,False,None,[10749],en,Heart Shot,Teenagers Nikki and Sam are in love and planni...,3.082,None,2022-02-17,Heart Shot,False,0.0,0,PG-13
3,920345,False,/qQFvUg92Uh1jRXCNgqo9LduBDFD.jpg,[18],en,EMPATHY (Or: The Girl with The Pearled Hair),After running away from her emotionally abusiv...,4.915,/59C1l5dqtKuKuKy5qlE23PV4yC6.jpg,2022-01-25,EMPATHY (Or: The Girl with The Pearled Hair),False,0.0,0,NR
4,641934,False,/rzu9BVI9HqY5rG0Xq05ZhewdkYP.jpg,"[18, 35]",en,Am I Ok?,Lucy and Jane have been best friends for most ...,9.426,None,2022-01-24,Am I Ok?,False,7.8,2,NR


### Drop columns from main dataframe

In [26]:
# Drop unnecessary columns from lgbt_movies_df dataframe.
lgbt_movies_df = lgbt_movies_df.drop(columns=[
    "adult", "backdrop_path", "original_language", "poster_path", "video", "vote_average", "vote_count"])

In [27]:
# Verify columns dropped and index reset correctly.
lgbt_movies_df.loc[[0, 359, 791]]

,id,genre_ids,original_title,overview,popularity,release_date,title,rating
0,507903,"[35, 18]",Spoiler Alert,The story of Michael Ausiello and Kit Cowan's ...,3.483,2022-12-31,Spoiler Alert,NR
359,449176,"[35, 18, 10749]","Love, Simon",Everyone deserves a great love story. But for ...,45.628,2018-03-16,"Love, Simon",PG-13
791,84892,[18],The Perks of Being a Wallflower,"Pittsburgh, Pennsylvania, 1991. High school fr...",36.050,2012-09-21,The Perks of Being a Wallflower,PG-13


## Gather Genre Info

### Create dataframe to hold genre names based on genre IDs

In [28]:
# Create genre_url variable
genre_url = "https://api.themoviedb.org/3/genre/movie/list?api_key=" + tmdb_api_key + "&language=en-US"

In [29]:
# Get the genre JSON info into a list.
genre_request = requests.get(genre_url)
genre_data = genre_request.json()
genre_info = genre_data["genres"]
genre_info

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [30]:
# Transform the list of dictionaries of genre information into a individual lists.
ids = [i["id"] for i in genre_info]
names = [n["name"] for n in genre_info]

print(ids)
print(names)

[28, 12, 16, 35, 80, 99, 18, 10751, 14, 36, 27, 10402, 9648, 10749, 878, 10770, 53, 10752, 37]
['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']


In [31]:
# Combine the lists "ids" and "names" into a single dictionary named "genre_dict".
genres_dict = {"ids": ids, "names": names}

### Duplicate genre_ids column into a new dataframe

In [32]:
# Duplicate genre_ids column into new dataframe.
genres_df = pd.DataFrame()
genres_df["genre"] = lgbt_movies_df['genre_ids']
genres_df.head()

,genre
0,"[35, 18]"
1,"[27, 53]"
2,[10749]
3,[18]
4,"[18, 35]"


In [33]:
# Separate the lists of genres in the genre column to hold one individual genre per column
genres_df = pd.DataFrame(
    genres_df['genre'].to_list(), columns=['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6']
)

In [34]:
# Verify the columns split.
genres_df.head()

,genre1,genre2,genre3,genre4,genre5,genre6
0,35.0,18.0,NaN,NaN,NaN,NaN
1,27.0,53.0,NaN,NaN,NaN,NaN
2,10749.0,NaN,NaN,NaN,NaN,NaN
3,18.0,NaN,NaN,NaN,NaN,NaN
4,18.0,35.0,NaN,NaN,NaN,NaN


### Transform Genre IDs into Genre Names.

In [35]:
# Replace all genre integer values for "genre_ids" to their associated string in genres_df.
genres_df = genres_df.replace(ids, names)

In [36]:
# Verify that genre names were replaced.
genres_df.head()

,genre1,genre2,genre3,genre4,genre5,genre6
0,Comedy,Drama,NaN,NaN,NaN,NaN
1,Horror,Thriller,NaN,NaN,NaN,NaN
2,Romance,NaN,NaN,NaN,NaN,NaN
3,Drama,NaN,NaN,NaN,NaN,NaN
4,Drama,Comedy,NaN,NaN,NaN,NaN


In [37]:
# Create a new column that puts all genre names in every column of the dataframe into a list per movie.
genres_df['genres'] = genres_df.values.tolist()

In [38]:
# Check work.
genres_df.head()

,genre1,genre2,genre3,genre4,genre5,genre6,genres
0,Comedy,Drama,NaN,NaN,NaN,NaN,"[Comedy, Drama, nan, nan, nan, nan]"
1,Horror,Thriller,NaN,NaN,NaN,NaN,"[Horror, Thriller, nan, nan, nan, nan]"
2,Romance,NaN,NaN,NaN,NaN,NaN,"[Romance, nan, nan, nan, nan, nan]"
3,Drama,NaN,NaN,NaN,NaN,NaN,"[Drama, nan, nan, nan, nan, nan]"
4,Drama,Comedy,NaN,NaN,NaN,NaN,"[Drama, Comedy, nan, nan, nan, nan]"


In [39]:
# Remove the null values from the list.
genres_df['genres'] = genres_df.genres.apply(lambda x: [name for name in x if not pd.isnull(name)])

In [40]:
# Check work.
genres_df.head()

,genre1,genre2,genre3,genre4,genre5,genre6,genres
0,Comedy,Drama,NaN,NaN,NaN,NaN,"[Comedy, Drama]"
1,Horror,Thriller,NaN,NaN,NaN,NaN,"[Horror, Thriller]"
2,Romance,NaN,NaN,NaN,NaN,NaN,[Romance]
3,Drama,NaN,NaN,NaN,NaN,NaN,[Drama]
4,Drama,Comedy,NaN,NaN,NaN,NaN,"[Drama, Comedy]"


In [41]:
# Drop individual genre columns.
genres_df = genres_df.drop(columns=['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6'])

In [42]:
# Check work.
genres_df.head()

,genres
0,"[Comedy, Drama]"
1,"[Horror, Thriller]"
2,[Romance]
3,[Drama]
4,"[Drama, Comedy]"


### Add Genre Names to LGBT movies dataframe.

In [43]:
# Add the genres column from genres_df into lgbt_movies_df.
lgbt_movies_df["genre"] = genres_df['genres']
lgbt_movies_df.head()

,id,genre_ids,original_title,overview,popularity,release_date,title,rating,genre
0,507903,"[35, 18]",Spoiler Alert,The story of Michael Ausiello and Kit Cowan's ...,3.483,2022-12-31,Spoiler Alert,NR,"[Comedy, Drama]"
1,876802,"[27, 53]",Saint Drogo,Saint Drogo is a supernatural slowburn themati...,1.070,2022-10-01,Saint Drogo,R,"[Horror, Thriller]"
2,929477,[10749],Heart Shot,Teenagers Nikki and Sam are in love and planni...,3.082,2022-02-17,Heart Shot,PG-13,[Romance]
3,920345,[18],EMPATHY (Or: The Girl with The Pearled Hair),After running away from her emotionally abusiv...,4.915,2022-01-25,EMPATHY (Or: The Girl with The Pearled Hair),NR,[Drama]
4,641934,"[18, 35]",Am I Ok?,Lucy and Jane have been best friends for most ...,9.426,2022-01-24,Am I Ok?,NR,"[Drama, Comedy]"


In [44]:
# Set up the new column order.
new_columns = [
    'id', 'genre_ids', 'genre', 'original_title', 'rating', 'overview', 'popularity', 'release_date', 'title']

In [45]:
# Reorder columns in lgbt_movies_df.
lgbt_movies_df = lgbt_movies_df[new_columns]

In [46]:
# Check work.
lgbt_movies_df.head()

,id,genre_ids,genre,original_title,rating,overview,popularity,release_date,title
0,507903,"[35, 18]","[Comedy, Drama]",Spoiler Alert,NR,The story of Michael Ausiello and Kit Cowan's ...,3.483,2022-12-31,Spoiler Alert
1,876802,"[27, 53]","[Horror, Thriller]",Saint Drogo,R,Saint Drogo is a supernatural slowburn themati...,1.070,2022-10-01,Saint Drogo
2,929477,[10749],[Romance],Heart Shot,PG-13,Teenagers Nikki and Sam are in love and planni...,3.082,2022-02-17,Heart Shot
3,920345,[18],[Drama],EMPATHY (Or: The Girl with The Pearled Hair),NR,After running away from her emotionally abusiv...,4.915,2022-01-25,EMPATHY (Or: The Girl with The Pearled Hair)
4,641934,"[18, 35]","[Drama, Comedy]",Am I Ok?,NR,Lucy and Jane have been best friends for most ...,9.426,2022-01-24,Am I Ok?
